In [361]:
import sqlite3 as sq
from pathlib import Path
from contextlib import closing

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import pyarrow as pa
import pyarrow.csv as pacsv

In [362]:
# define some constants and such
key_cols = dict(idx='INT NOT NULL', timestamp='TEXT NOT NULL', stat='TEXT NOT NULL')
instr_sns = [i for i in range(-10_000, 0)]
logger_sns = [i for i in range(-10_000, 0)]

In [363]:
def create_table(name, columns, con, csr, primary=None, foreign=None):
    '''
    creates a sqlite3 table
    csr, con: cursor and connection objects
    name: the table name
    columns: a dict of column names as {colname:dtype}
    primary: list of str indicating primary keys
    foreign: list of tuples the form (foreign_key_name, ref_table_name, ref_primary_key_name) indicating foreign key links
    '''
    
    columns = [f'{name} {_type}' for name, _type in columns.items()]
    columns = ", ".join(columns)
    query = f'CREATE TABLE IF NOT EXISTS {name}({columns})'
    
    if primary is not None:
        query += f', PRIMARY KEY({", ".join(primary)})'
        
    if foreign is not None:
        for foreign_key, ref_table, ref_key in foreign:
            query += f', FOREIGN KEY ({foreign}) REFERENCES {ref_table} ({ref_primary}) ON DELETE CASCADE ON UPDATE NO ACTION'
    
    print(query, '\n')
    
    # send the query
    csr.execute(query)
    # save changes
    con.commit()
    
    return

def insert(name, values, columns=None, con=None, csr=None):
    '''
    inserts several rows of data
    csr, con: cursor and connection objects
    name: table to insert into
    values: list of tuples, shape (nrows, ncols)
    columns: iterable of column names for associated values
    '''
    
    # create a string '?, ?, ?, ...' 
    qmarks = ', '.join(['?']*len(values[0]))
    if columns is not None:
        # create a string 'col1, col2, col3, ...'
        columns = ', '.join(columns)
        
        query = f'INSERT INTO {name}({columns}) VALUES({qmarks})'
    
    else:
        query = f'INSERT INTO {name} VALUES({qmarks})'
    
    print(query, '\n')
    csr.executemany(query, values)
    con.commit()
    
    return

def create_multi_index(name, columns, con, csr):
    '''create a multi-column index on the given columns. columns is a list of str.'''
    
    columns = ', '.join(columns)
    query = f'CREATE INDEX IF NOT EXISTS {name}_multidx ON {name}({columns});'
    print(query, '\n')
    
    csr.execute(query)
    
    con.commit()
    
    
    return


    
def add_instrument(instr_model, site, height, rep, instr_sn, logger_sn, shortname, comment, columns, units, con, show):
    '''
    Add a new instrument. This will create a  new data table for this instrument, record instrument metadata in the associated instrument metadata table, and add units to a unit table.
    instr_model: str, model name
    site: str, site name (without height)
    rep: int, indicate uniqueness of this instrument/type of instrument
    instr_sn: int or bool, if not given, a unique negative integer will be chosen.
    logger_sn: int, serial number of logger attached to instrument
    shortname: str, should be short and convey instrument type. examples: irga, son, irgason, thp, soiltdr
    columns: dict of {variable:datatype} for data columns, not including timestamp or record numbers
    units: dict of {variable:units} for data columns, not including timestamp or record numbers. Use the format <unit>±n<unit>±m, for example m+1s-1, or C+1.
    comment: str, any additional info about the instrument.
    con: sqlite3.connect(), a connection to the sqlite3 database
    show: bool (default False), whether to display the created/modified tables.
    '''
    
    csr = con.cursor()
    
    # get serial numbers: assign instr serial number randomely if not given
    if not instr_sn:
        instr_sn = instr_sns.pop()
    
    # create data table for instrument
    columns.update(key_cols)
    instr_table_name = f'{shortname}_{site}_{height}_{rep}'
    create_table(instr_table_name, columns, con, csr)
    create_multi_index(instr_table_name, ['idx', 'timestamp', 'stat'], con, csr)
    
    # create/update units table
    insert('units', [(instr_sn, var_name, unit, '') for var_name, unit in units.items()], con=con, csr=csr)
    
    # add metadata to logger table
    insert('instruments', [(shortname, instr_model, site, height, rep, instr_sn, logger_sn, instr_table_name, comment)], con=con, csr=csr)

    if show:
        print(instr_table_name)
        display(pd.read_sql(f'SELECT * FROM {instr_table_name}', con))
        print('instruments')
        display(pd.read_sql(f'SELECT * FROM instruments', con))
        print('units')
        display(pd.read_sql(f'SELECT * FROM units', con))
        
    return

def add_logger(logger_model, site, rep, logger_sn, shortname, comment, con, show):
    '''
    Add a new instrument. This will create a  new data table for this instrument, record instrument metadata in the associated instrument metadata table, and add units to a unit table.
    logger_model: str, logger model
    site: str, site name (without height)
    rep: int, indicate uniqueness of this instrument/type of instrument
    logger_sn: int, serial number of logger attached to instrument
    shortname: str, should be short and convey instrument type. examples: irga, son, irgason, thp, soiltdr
    comment: str, any additional info about the instrument.
    con: sqlite3.connect(), a connection to the sqlite3 database
    show: bool (default False), whether to display the created/modified tables.
    '''
    
    csr = con.cursor()
    
    # get serial numbers: assign instr serial number randomely if not given
    if not logger_sn:
        logger_sn = logger_sns.pop()
    print(logger_sn)
    
    # add metadata to logger table
    insert('loggers', [(site, logger_model, logger_sn, shortname, comment)], con=con, csr=csr)

    if show:
        print('loggers')
        display(pd.read_sql(f'SELECT * FROM loggers', con))
        
    return

# Input data locations

In [369]:
sandbox_dir = Path('/Volumes/TempData/Bretfeld Mario/Chimney-Park-Reprocessing-Sandbox/')
data_dir = Path('/Volumes/TempData/Bretfeld Mario/Chimney') / 'Data'
wd = sandbox_dir = sandbox_dir/'Alex Work'
db_fn = Path('/Users/waldinian/Documents/CPTest6.db')

# Initialize main database structure

In [370]:
with closing(sq.connect(db_fn)) as con:
    csr = con.cursor()
    
    # create logger table
    csr.execute(
        '''
        CREATE TABLE IF NOT EXISTS loggers(
            site TEXT NOT NULL,
            logger_model TEXT NOT NULL,
            logger_sn INT NOT NULL,
            shortname TEXT NOT NULL,
            comment TEXT,
            PRIMARY KEY (logger_sn)
        );
        '''
    )
    csr.execute('''CREATE INDEX IF NOT EXISTS loggers_snidx ON loggers(logger_sn);''')
    
    # create instrument table
    csr.execute(
        '''
        CREATE TABLE IF NOT EXISTS instruments(
            shortname TEXT NOT NULL,
            instr_model TEXT NOT NULL,
            site TEXT NOT NULL,
            height INT NOT NULL,
            rep INT NOT NULL,
            instr_sn INT NOT NULL,
            logger_sn INT NOT NULL,
            instr_table TEXT NOT NULL,
            comment TEXT,
            PRIMARY KEY(instr_sn),
            FOREIGN KEY(logger_sn) REFERENCES loggers(logger_sn) 
                ON DELETE CASCADE 
                ON UPDATE NO ACTION
        );
        '''
    )
    csr.execute('''CREATE INDEX IF NOT EXISTS instruments_snidx ON instruments(instr_sn);''')
    csr.execute('''CREATE INDEX IF NOT EXISTS instruments_nameidx ON instruments(shortname);''')
    csr.execute('''CREATE INDEX IF NOT EXISTS instruments_tableidx ON instruments(instr_table);''')
    
    # units table
    csr.execute(
        '''
        CREATE TABLE IF NOT EXISTS units(
            instr_sn TEXT NOT NULL,
            variable TEXT NOT NULL,
            units TEXT NOT NULL,
            comment TEXT,
            FOREIGN KEY(instr_sn) REFERENCES instruments(instr_sn) 
                ON DELETE CASCADE 
                ON UPDATE NO ACTION
            
            )
        '''
    )
    csr.execute('''CREATE INDEX IF NOT EXISTS units_multidx ON units(instr_sn, variable)''')
    
    
    con.commit()

# Create instrument tables

In [371]:
with closing(sq.connect(db_fn)) as con:
        
        # slow data
        sonic_cols = dict(u='REAL', v='REAL', w='REAL', t_sonic='REAL', diag_sonic='INT')
        irga_cols = dict(co2='REAL', h2o='REAL', p_irga='REAL', diag_irga='INT')
        irgason_cols = dict(u='REAL', v='REAL', w='REAL', t_sonic='REAL', diag_sonic='INT', co2='REAL', h2o='REAL', p_irga='REAL', t_irga='REAL', diag_irga='INT', co2_strength='REAL', h2o_strength='REAL')
        status_cols = dict(v_logger='REAL', t_logger='REAL')
        
        sonic_units = dict(u='m+1s-1', v='m+1s-1', w='m+1s-1', t_sonic='C+1', diag_sonic='')
        irga_units = dict(co2='mg+1m-3', h2o='g+1m-3', p_irga='kPa+1', diag_irga='')
        irgason_units = dict(u='m+1s-1', v='m+1s-1', w='m+1s-1', t_sonic='C+1', diag_sonic='', co2='mg+1m-3', h2o='g+1m-3', p_irga='kPa+1', t_irga='C+1', diag_irga='', co2_strength='', h2o_strength='')
        status_units = dict(v_logger='V+1', t_logger='C+1')
        
        show=False
        
        # add EC logger metadata
        add_logger(site='NF', logger_model='CR3000', rep=1, logger_sn='9809', shortname='NF EC 17m', con=con, show=show, comment='')
        add_logger(site='NF', logger_model='CR3000', rep=2, logger_sn='10365', shortname='NF EC 3m', con=con, show=show, comment='')
        add_logger(site='SF', logger_model='CR6', rep=1, logger_sn='2992', shortname='SF EC 4m', con=con, show=show, comment='')
        add_logger(site='SF', logger_model='CR3000', rep=2, logger_sn='9810', shortname='SF EC 7m', con=con, show=show, comment='')
        add_logger(site='UF', logger_model='CR6', rep=1, logger_sn='2991', shortname='UF EC 3m', con=con, show=show, comment='')
        
        # add EC logger status data tables
        add_instrument(shortname='status', site='NF', height=0, instr_model='CR3000', rep=1, instr_sn=9809, 
                       comment='NF EC 17m logger', columns=status_cols, units=status_units, logger_sn=9809, con=con, show=show)
        add_instrument(shortname='status', site='NF', height=0, instr_model='CR3000', rep=2, instr_sn=10365, 
                       comment='NF EC 3m logger', columns=status_cols, units=status_units, logger_sn=10365, con=con, show=show)
        add_instrument(shortname='status', site='SF', height=0, instr_model='CR6', rep=1, instr_sn=2992, 
                       comment='SF EC 4m logger', columns=status_cols, units=status_units, logger_sn=2992, con=con, show=show)
        add_instrument(shortname='status', site='SF', height=0, instr_model='CR3000', rep=2, instr_sn=9810, 
                       comment='SF EC 7m logger', columns=status_cols, units=status_units, logger_sn=9810, con=con, show=show)
        add_instrument(shortname='status', site='UF', height=0, instr_model='CR6', rep=1, instr_sn=2991, 
                       comment='UF EC 3m logger', columns=status_cols, units=status_units, logger_sn=2991, con=con, show=show)
        
        
        # add EC instrument data tables
        add_instrument(shortname='sonic', site='NF', height=17, instr_model='CSAT3', rep=1, instr_sn=False, comment='3d sonic', columns=sonic_cols, units=sonic_units, logger_sn=9809, con=con, show=show)
        add_instrument(shortname='sonic', site='NF', height=7, instr_model='CSAT3', rep=1, instr_sn=False, comment='3d sonic', columns=sonic_cols, units=sonic_units, logger_sn=9809, con=con, show=show)
        add_instrument(shortname='sonic', site='NF', height=3, instr_model='CSAT3B', rep=1, instr_sn=False, comment='3d sonic', columns=sonic_cols, units=sonic_units, logger_sn=10365, con=con, show=show)
        add_instrument(shortname='sonic', site='SF', height=7, instr_model='CSAT3B', rep=1, instr_sn=False, comment='3d sonic', columns=sonic_cols, units=sonic_units, logger_sn=9810, con=con, show=show)
        add_instrument(shortname='irga', site='NF', height=17, instr_model='LI7500', rep=1, instr_sn=False, comment='3d sonic', columns=irga_cols, units=irga_units, logger_sn=9809, con=con, show=show)
        add_instrument(shortname='irga', site='NF', height=3, instr_model='LI7500', rep=1, instr_sn=False, comment='3d sonic', columns=irga_cols, units=irga_units, logger_sn=10365, con=con, show=show)
        add_instrument(shortname='irga', site='SF', height=7, instr_model='LI7500', rep=1, instr_sn=False, comment='3d sonic', columns=irga_cols, units=irga_units, logger_sn=9810, con=con, show=show)
        add_instrument(shortname='irgason', site='SF', height=4, instr_model='IRGASON', rep=1, instr_sn=False, comment='3d sonic and co2 and h2o', columns=irgason_cols, units=irgason_units, logger_sn=2992, con=con, show=show)
        add_instrument(shortname='irgason', site='UF', height=3, instr_model='IRGASON', rep=1, instr_sn=False, comment='3d sonic and co2 and h2o', columns=irgason_cols, units=irgason_units, logger_sn=2991, con=con, show=show)

9809
INSERT INTO loggers VALUES(?, ?, ?, ?, ?) 

10365
INSERT INTO loggers VALUES(?, ?, ?, ?, ?) 

2992
INSERT INTO loggers VALUES(?, ?, ?, ?, ?) 

9810
INSERT INTO loggers VALUES(?, ?, ?, ?, ?) 

2991
INSERT INTO loggers VALUES(?, ?, ?, ?, ?) 

CREATE TABLE IF NOT EXISTS status_NF_0_1(v_logger REAL, t_logger REAL, idx INT NOT NULL, timestamp TEXT NOT NULL, stat TEXT NOT NULL) 

CREATE INDEX IF NOT EXISTS status_NF_0_1_multidx ON status_NF_0_1(idx, timestamp, stat); 

INSERT INTO units VALUES(?, ?, ?, ?) 

INSERT INTO instruments VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?) 

CREATE TABLE IF NOT EXISTS status_NF_0_2(v_logger REAL, t_logger REAL, idx INT NOT NULL, timestamp TEXT NOT NULL, stat TEXT NOT NULL) 

CREATE INDEX IF NOT EXISTS status_NF_0_2_multidx ON status_NF_0_2(idx, timestamp, stat); 

INSERT INTO units VALUES(?, ?, ?, ?) 

INSERT INTO instruments VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?) 

CREATE TABLE IF NOT EXISTS status_SF_0_1(v_logger REAL, t_logger REAL, idx INT NOT NULL, timestamp TEX

# Add data to the tables

In [372]:
def upload_slowflux(db_fn):
    with closing(sq.connect(db_fn)) as con:
        
#         slowflux_fns = list(data_dir.glob('BB*/EC/*m/20*/Converted/TOA5*Flux30Min.dat'))
        slowflux_fns = [
            '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20201017/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20201104/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190504/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200610/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190320/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210602/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190926/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210520/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190327/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20201217/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191107/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191219/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200215/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210330/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20201124/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191011/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210106/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190813/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190725/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20181206/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200306/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190519/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200116/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190711/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210813/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210112/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190115/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190227/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190903/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190406/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200224/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190222/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210712/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190216/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191114/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190625/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210420/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210124/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200527/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200313/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191031/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200803/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210119/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210621/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200820/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190701/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190910/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200912/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190822/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200721/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191121/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190514/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190508/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20181201/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191016/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20210306/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191002/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200626/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20191205/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20181129/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190919/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190620/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190420/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20201112/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200901/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190603/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20200710/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/7m/20190806/Converted/TOA5_9810.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20201017/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20201104/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190504/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200610/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190320/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190926/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190327/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20201217/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191107/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191219/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200215/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210330/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20201124/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191011/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210106/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190813/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20181206/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200306/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190519/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200116/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190711/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210813/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210112/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190115/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190227/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190903/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190406/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200224/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190222/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210712/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190216/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191114/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190625/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210124/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200527/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191031/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200803/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210119/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210621/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200820/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190701/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190910/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200912/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190822/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200721/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191121/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190514/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190508/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191016/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191002/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200626/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20191205/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190919/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190620/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190420/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20201112/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200901/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20210308/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190603/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20200710/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-SF/EC/4m/20190806/Converted/TOA5_2992.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20201017/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20201104/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190504/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200610/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210602/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190926/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210520/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20201217/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191107/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191219/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200215/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210330/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191011/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210106/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190813/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190725/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200306/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190519/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200116/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190711/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210112/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200224/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20201114/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210712/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191114/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190625/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210420/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210124/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200527/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200313/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191031/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200803/Converted/TOA5_2221.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210119/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210621/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200820/Converted/TOA5_2221.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190701/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190910/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200912/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200721/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191121/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190514/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190508/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191016/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20210306/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191002/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200626/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20191205/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190919/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190620/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20201112/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200901/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190603/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20200710/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/17m/20190806/Converted/TOA5_9809.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20201017/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20201104/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190504/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20211208/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200610/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210602/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190926/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210520/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20201217/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191107/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191219/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200215/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210330/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20201124/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191011/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210106/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190813/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190725/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200306/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190519/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200116/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190711/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210930/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210813/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210112/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190903/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200224/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210712/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191114/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190625/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20211105/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210420/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210124/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200527/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200313/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191031/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200803/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210119/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210621/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200820/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190701/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190910/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200912/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190822/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20211220/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200721/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191121/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190514/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190508/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191016/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20210306/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191002/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200626/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20191205/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190919/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190620/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190420/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20201112/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200901/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190603/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20200710/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-NF/EC/3m/20190806/Converted/TOA5_10365.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20201017/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20201104/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190504/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200610/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210602/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190926/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20201217/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191107/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210330/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20201124/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210106/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190813/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190725/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200306/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190519/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200116/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190711/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210813/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210112/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190903/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190406/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210712/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191114/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190625/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210124/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200313/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191031/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200803/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210119/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20210621/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200820/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190701/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190910/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200912/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190822/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191121/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190602/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190514/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190508/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191016/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191002/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200626/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20191205/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190919/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190620/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20190420/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20201112/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200901/Converted/TOA5_2991.Flux30Min.dat', '/Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200710/Converted/TOA5_2991.Flux30Min.dat'
        ]
    
        pbar = tqdm(slowflux_fns)
        for fn in pbar:
            fn = Path(fn)
            pbar.set_description(f'Processing {fn}')
            
            site = fn.parent.parent.parent.parent.parent.name
            height = fn.parent.parent.parent.name
            site = str(site) + str(height)
            if site=='BB-NF17m':
                # read in file
                
                df = pd.read_csv(fn, skiprows=[0, 2, 3])
                
                # rename to standardized names
                cols_to_drop = ['RECORD', 'n_Tot']
                renaming_dict = {
                    'TIMESTAMP':'timestamp', 
                    'Ux_CSAT3_17m_Avg': 'u.17-avg',
                    'Uy_CSAT3_17m_Avg': 'v.17-avg',
                    'Uz_CSAT3_17m_Avg': 'w.17-avg',
                    'Ts_CSAT3_17m_Avg': 't_sonic.17-avg',
                    'Ux_CSAT3_7m_Avg': 'u.7-avg',
                    'Uy_CSAT3_7m_Avg': 'v.7-avg',
                    'Uz_CSAT3_7m_Avg': 'w.7-avg',
                    'Ts_CSAT3_7m_Avg': 't_sonic.7-avg',
                    'rho_c_LI7500_Avg': 'co2.17-avg',
                    'rho_v_LI7500_Avg': 'h2o.17-avg',
                    'P_LI7500_Avg': 'p_irga.17-avg',
                    'DIAG_CSAT3_17m_Avg': 'diag_sonic.17-avg',
                    'DIAG_LI7500_Avg': 'diag_irga.17-avg',
                    'T_CR3000_Avg': 't_logger.0-avg',
                    'V_CR3000_Avg': 'v_logger.0-avg',
                    'Ux_CSAT3_17m_Std': 'u.17-std',
                    'Uy_CSAT3_17m_Std': 'v.17-std',
                    'Uz_CSAT3_17m_Std': 'w.17-std',
                    'Ts_CSAT3_17m_Std': 't_sonic.17-std',
                    'Ux_CSAT3_7m_Std': 'u.7-std',
                    'Uy_CSAT3_7m_Std': 'v.7-std',
                    'Uz_CSAT3_7m_Std': 'w.7-std',
                    'Ts_CSAT3_7m_Std': 't_sonic.7-std',
                    'rho_c_LI7500_Std': 'co2.17-std',
                    'rho_v_LI7500_Std': 'h2o.17-std',
                    'P_LI7500_Std': 'p_irga.17-std',
                    'DIAG_CSAT3_17m_Std': 'diag_sonic.17-std',
                    'DIAG_CSAT3_7m_Std': 'diag_sonic.7-std',
                    'DIAG_LI7500_Std': 'diag_irga.17-std',
                    'T_CR3000_Std': 't_logger.0-std',
                    'V_CR3000_Std': 'v_logger.0-std'}
                df = (df
                      .drop(columns=cols_to_drop)
                      .rename(renaming_dict, axis='columns'))
                
                # the above dictionary contains all possible columns. Remove the ones that we don't see in this file.
                desired_columns_sonic17 = ['timestamp', 'u.17-avg', 'v.17-avg', 'w.17-avg', 't_sonic.17-avg', 'diag_sonic.17-avg', 'u.17-std', 'v.17-std', 'w.17-std', 't_sonic.17-std']
                desired_columns_sonic7 = ['timestamp', 'u.7-avg', 'v.7-avg', 'w.7-avg', 't_sonic.7-avg', 'diag_sonic.7-avg', 'u.7-std', 'v.7-std', 'w.7-std', 't_sonic.7-std']
                desired_columns_irga17 = ['timestamp', 'co2.17-avg', 'h2o.17-avg', 'p_irga.17-avg', 'diag_irga.17-avg', 'co2.17-std', 'h2o.17-std', 'p_irga.17-std', 'diag_irga.17-std']
                desired_columns_status = ['timestamp', 't_logger.0-avg', 'v_logger.0-avg', 't_logger.0-std', 'v_logger.0-std']
                for i, desired_column in enumerate(desired_columns_sonic17):
                    if desired_column not in df.columns:
                        desired_columns_sonic17.pop(i)
                for i, desired_column in enumerate(desired_columns_sonic7):
                    if desired_column not in df.columns:
                        desired_columns_sonic7.pop(i)
                for i, desired_column in enumerate(desired_columns_irga17):
                    if desired_column not in df.columns:
                        desired_columns_irga17.pop(i)
                for i, desired_column in enumerate(desired_columns_status):
                    if desired_column not in df.columns:
                        desired_columns_status.pop(i)
                        
                # subset the data by instrument
                sonic_17 = df[desired_columns_sonic17]
                sonic_7 = df[desired_columns_sonic7]
                irga_17 = df[desired_columns_irga17]
                status = df[desired_columns_status]

                # pivot longer by expanding on height and stat suffixes
                ds = [sonic_17, sonic_7, irga_17, status]
                for di, d in enumerate(ds):
                    # turn all columns but 'timestamp' into stubs
                    cols_to_stub = d.drop(columns='timestamp').columns
                    # first: pass make stat column, index by timestamp, chop off at '-'.
                    # second pass: make height column, index by timestamp and stat, chop off at '.'
                    i = [['timestamp', 'index'], ['timestamp', 'index', 'stat']]
                    j = ['stat', 'height']
                    sep = ['-', '.']
                    suffix = ['\w+', '\d+']
                    # generate stubnames
                    stubnames = [[*set([colname.split(sep[0])[0] for colname in cols_to_stub])],
                                 [*set([colname.split(sep[1])[0] for colname in cols_to_stub])]]
                    
                    # add column 'index'
                    d = d.reset_index()
                    # pivot wider
                    for k in range(2):
                        d = pd.wide_to_long(d, stubnames[k], i=i[k], j=j[k], sep=sep[k], suffix=suffix[k])
                        # wide_to_long will turn columns i and j into index columns. Undo this so we can pivot again.
                        d=d.reset_index()
                    # remove height columns
                    d = d.drop(columns='height').rename({'index':'idx'}, axis='columns')
                    # overwrite old list entry
                    ds[di] = d
                
                # upload (append) to database
                sonic_17 = ds[0]
                sonic_7 = ds[1]
                irga_17 = ds[2]
                status = ds[3]
                
                sonic_17.to_sql('sonic_NF_17_1', con, if_exists='append', index=False)
                sonic_7.to_sql('sonic_NF_7_1', con, if_exists='append', index=False)
                irga_17.to_sql('irga_NF_17_1', con, if_exists='append', index=False)
                status.to_sql('status_NF_17_1', con, if_exists='append', index=False)
                
#             if site=='BB-NF3m':
#                 # read in file
#                 df = pd.read_csv(fn, skiprows=[0, 2, 3])
                
#                 # rename to standardized names
#                 cols_to_drop = ['RECORD', 'n_Tot']
#                 renaming_dict = {
#                     'TIMESTAMP':'timestamp', 
#                     'Ux_CSAT3B_Avg': 'u.3-avg',
#                     'Uy_CSAT3B_Avg': 'v.3-avg',
#                     'Uz_CSAT3B_Avg': 'w.3-avg',
#                     'Ts_CSAT3B_Avg': 't_sonic.3-avg',
#                     'rho_c_LI7500_Avg': 'co2.3-avg',
#                     'rho_v_LI7500_Avg': 'h2o.3-avg',
#                     'P_LI7500_Avg': 'p_irga.3-avg',
#                     'DIAG_CSAT3B_Avg': 'diag_sonic.3-avg',
#                     'DIAG_LI7500_Avg': 'diag_irga.3-avg',
#                     'T_CR3000_Avg': 't_logger.0-avg',
#                     'V_CR3000_Avg': 'v_logger.0-avg',
#                     'Ux_CSAT3B_Std': 'u.3-std',
#                     'Uy_CSAT3B_Std': 'v.3-std',
#                     'Uz_CSAT3B_Std': 'w.3-std',
#                     'Ts_CSAT3B_Std': 't_sonic.3-std',
#                     'rho_c_LI7500_Std': 'co2.3-std',
#                     'rho_v_LI7500_Std': 'h2o.3-std',
#                     'P_LI7500_Std': 'p_irga.3-std',
#                     'DIAG_CSAT3B_Std': 'diag_sonic.3-std',
#                     'DIAG_LI7500_Std': 'diag_irga.3-std',
#                     'T_CR3000_Std': 't_logger.0-std',
#                     'V_CR3000_Std': 'v_logger.0-std'}
#                 df = df.drop(columns=cols_to_drop).rename(renaming_dict, axis='columns')
                
#                 # subset the data by instrument
#                 sonic_cols = ['timestamp', 'u.3-avg', 'v.3-avg', 'w.3-avg', 't_sonic.3-avg', 'diag_sonic.3-avg', 'u.3-std', 'v.3-std', 'w.3-std', 't_sonic.3-std', 'diag_sonic.3-std']
#                 irga_cols = ['timestamp', 'co2.3-avg', 'h2o.3-avg', 'p_irga.3-avg', 'diag_irga.3-avg', 'co2.3-std', 'h2o.3-std', 'p_irga.3-std', 'diag_irga.3-std']
#                 logger_cols = ['timestamp', 't_logger.0-avg', 'v_logger.0-avg', 't_logger.0-std', 'v_logger.0-std']
                
#                 # pivot longer by expanding on height and stat suffixes
#                 ds = [df[sonic_cols], df[irga_cols], df[logger_cols]]

#                 for di, d in enumerate(ds):
#                     cols_to_stub = d.drop(columns='timestamp')
#                     i = ['timestamp', ['timestamp', 'stat']]
#                     j = ['stat', 'height']
#                     sep = ['-', '.']
#                     suffix = ['\w+', '\d+']
#                     stubnames = [[*set([colname.split(sep[0])[0] for colname in cols_to_stub])],
#                                  [*set([colname.split(sep[1])[0] for colname in cols_to_stub])]]
#                     for k in range(2):
#                         d = pd.wide_to_long(d, stubnames[k], i=i[k], j=j[k], sep=sep[k], suffix=suffix[k]).reset_index()
#                     d['site'] = 'NF3'
#                     d['replicate'] = 1

#                     ds[di] = d
                
#                 # upload (append) to database
#                 sonic = ds[0]
#                 irga = ds[1]
#                 logger = ds[2]

#                 sonic.to_sql('sonic_slow', con, if_exists='append', index=False)
#                 irga.to_sql('irga_slow', con, if_exists='append', index=False)
#                 logger.to_sql('logger_slow', con, if_exists='append', index=False)
    
#             if site=='BB-SF4m':
#                 # read in file
#                 df = pd.read_csv(fn, skiprows=[0, 2, 3])
                
#                 # rename to standardized names
#                 cols_to_drop = ['RECORD', 'DT_Avg', 'DT_Std', 'Q_Avg', 'Q_Std', 'TCDT_Avg', 'TCDT_Std', 'DBTCDT_Avg', 'DBTCDT_Std', 'CO2_sig_strgth_Avg', 'H2O_sig_strgth_Avg', 'CO2_sig_strgth_Std', 'H2O_sig_strgth_Std',]
                
#                 renaming_dict = {
#                     'TIMESTAMP':'timestamp', 
#                     'Ux_Avg': 'u.4-avg',
#                     'Uy_Avg': 'v.4-avg',
#                     'Uz_Avg': 'w.4-avg',
#                     'Ts_Avg': 't_sonic.4-avg',
#                     'CO2_Avg': 'co2.4-avg',
#                     'H2O_Avg': 'h2o.4-avg',
#                     'cell_press_Avg': 'p_irga.4-avg',
#                     'cell_tmpr_Avg': 't_irga.4-avg',
#                     'diag_sonic_Avg': 'diag_sonic.4-avg',
#                     'diag_irga_Avg': 'diag_irga.4-avg',
#                     'PTemp_C_Avg': 't_logger.0-avg',
#                     'BattV_Avg': 'v_logger.0-avg',
#                     'BattV_Min': 'v_logger.0-min',
#                     'Ux_Std': 'u.4-std',
#                     'Uy_Std': 'v.4-std',
#                     'Uz_Std': 'w.4-std',
#                     'Ts_Std': 't_sonic.4-std',
#                     'CO2_Std': 'co2.4-std',
#                     'H2O_Std': 'h2o.4-std',
#                     'cell_press_Std': 'p_irga.4-std',
#                     'cell_tmpr_Std': 't_irga.4-std',
#                     'diag_irga_Std': 'diag_sonic.4-std',
#                     'diag_sonic_Std': 'diag_irga.4-std',
#                     }
#                 df = df.drop(columns=cols_to_drop).rename(renaming_dict, axis='columns')
                
#                 # subset the data by instrument
#                 sonic_cols = ['timestamp', 'u.4-avg', 'v.4-avg', 'w.4-avg', 't_sonic.4-avg', 'diag_sonic.4-avg', 'u.4-std', 'v.4-std', 'w.4-std', 't_sonic.4-std', 'diag_sonic.4-std']
#                 irga_cols = ['timestamp', 'co2.4-avg', 'h2o.4-avg', 'p_irga.4-avg', 't_irga.4-avg', 'diag_irga.4-avg', 'co2.4-std', 'h2o.4-std', 'p_irga.4-std', 'diag_irga.4-std']
#                 logger_cols = ['timestamp', 't_logger.0-avg', 'v_logger.0-avg', 'v_logger.0-min']
                
#                 # pivot longer by expanding on height and stat suffixes
#                 ds = [df[sonic_cols], df[irga_cols], df[logger_cols]]

#                 for di, d in enumerate(ds):
#                     cols_to_stub = d.drop(columns='timestamp')
#                     i = ['timestamp', ['timestamp', 'stat']]
#                     j = ['stat', 'height']
#                     sep = ['-', '.']
#                     suffix = ['\w+', '\d+']
#                     stubnames = [[*set([colname.split(sep[0])[0] for colname in cols_to_stub])],
#                                  [*set([colname.split(sep[1])[0] for colname in cols_to_stub])]]
#                     for k in range(2):
#                         d = pd.wide_to_long(d, stubnames[k], i=i[k], j=j[k], sep=sep[k], suffix=suffix[k]).reset_index()
#                     d['site'] = 'SF4'
#                     d['replicate'] = 1

#                     ds[di] = d
                
#                 # upload (append) to database
#                 sonic = ds[0]
#                 irga = ds[1]
#                 logger = ds[2]

#                 sonic.to_sql('sonic_slow', con, if_exists='append', index=False)
#                 irga.to_sql('irga_slow', con, if_exists='append', index=False)
#                 logger.to_sql('logger_slow', con, if_exists='append', index=False)
                
#             if site=='BB-SF7m':
#                 # read in file
#                 df = pd.read_csv(fn, skiprows=[0, 2, 3])
#                 print(df.columns)
#                 display(df)
                
#                 # rename to standardized names
#                 cols_to_drop = ['RECORD', 'n_Tot']
#                 renaming_dict = {
#                     'TIMESTAMP':'timestamp', 
#                     'Ux_CSAT3B_Avg': 'u.7-avg',
#                     'Uy_CSAT3B_Avg': 'v.7-avg',
#                     'Uz_CSAT3B_Avg': 'w.7-avg',
#                     'Ts_CSAT3B_Avg': 't_sonic.7-avg',
#                     'rho_c_LI7500_Avg': 'co2.7-avg',
#                     'rho_v_LI7500_Avg': 'h2o.7-avg',
#                     'P_LI7500_Avg': 'p_irga.7-avg',
#                     'DIAG_CSAT3B_Avg': 'diag_sonic.7-avg',
#                     'DIAG_LI7500_Avg': 'diag_irga.7-avg',
#                     'T_CR3000_Avg': 't_logger.0-avg',
#                     'V_CR3000_Avg': 'v_logger.0-avg',
#                     'Ux_CSAT3B_Std': 'u.7-std',
#                     'Uy_CSAT3B_Std': 'v.7-std',
#                     'Uz_CSAT3B_Std': 'w.7-std',
#                     'Ts_CSAT3B_Std': 't_sonic.7-std',
#                     'rho_c_LI7500_Std': 'co2.7-std',
#                     'rho_v_LI7500_Std': 'h2o.7-std',
#                     'P_LI7500_Std': 'p_irga.7-std',
#                     'DIAG_CSAT3B_Std': 'diag_sonic.7-std',
#                     'DIAG_LI7500_Std': 'diag_irga.7-std',
#                     'T_CR3000_Std': 't_logger.0-std',
#                     'V_CR3000_Std': 'v_logger.0-std'}
#                 df = df.drop(columns=cols_to_drop).rename(renaming_dict, axis='columns')
                
#                 # subset the data by instrument
#                 sonic_cols = ['timestamp', 'u.7-avg', 'v.7-avg', 'w.7-avg', 't_sonic.7-avg', 'diag_sonic.7-avg', 'u.7-std', 'v.7-std', 'w.7-std', 't_sonic.7-std', 'diag_sonic.7-std']
#                 irga_cols = ['timestamp', 'co2.7-avg', 'h2o.7-avg', 'p_irga.7-avg', 'diag_irga.7-avg', 'co2.7-std', 'h2o.7-std', 'p_irga.7-std', 'diag_irga.7-std']
#                 logger_cols = ['timestamp', 't_logger.0-avg', 'v_logger.0-avg', 't_logger.0-std', 'v_logger.0-std']
                
#                 # pivot longer by expanding on height and stat suffixes
#                 ds = [df[sonic_cols], df[irga_cols], df[logger_cols]]

#                 for di, d in enumerate(ds):
#                     d.reset_index(inplace=True)
#                     cols_to_stub = d.drop(columns='timestamp')
#                     i = [['timestamp', 'index'], ['timestamp', 'index', 'stat']]
#                     j = ['stat', 'height']
#                     sep = ['-', '.']
#                     suffix = ['\w+', '\d+']
                    
                    
#                     stubnames = [[*set([colname.split(sep[0])[0] for colname in cols_to_stub])],
#                                  [*set([colname.split(sep[1])[0] for colname in cols_to_stub])]]
#                     for k in range(2):
                        
#                         print(len(set(d.timestamp)))
#                         print(len(d.timestamp))
                        
#                         d = pd.wide_to_long(d, stubnames[k], i=i[k], j=j[k], sep=sep[k], suffix=suffix[k]).reset_index()
#                     d['site'] = 'SF7'
#                     d['replicate'] = 1

#                     ds[di] = d
                
#                 # upload (append) to database
#                 sonic = ds[0]
#                 irga = ds[1]
#                 logger = ds[2]

#                 sonic.to_sql('sonic_slow', con, if_exists='append', index=False)
#                 irga.to_sql('irga_slow', con, if_exists='append', index=False)
#                 logger.to_sql('logger_slow', con, if_exists='append', index=False)

#             if site=='BB-UF3m':
#                 # read in file
#                 df = pd.read_csv(fn, skiprows=[0, 2, 3])
#                 # rename to standardized names
#                 cols_to_drop = ['RECORD', 'CO2_sig_strgth_Avg', 'H2O_sig_strgth_Avg', 'CO2_sig_strgth_Std', 'H2O_sig_strgth_Std',]

                
#                 renaming_dict = {
#                     'TIMESTAMP':'timestamp', 
#                     'Ux_Avg': 'u.3-avg',
#                     'Uy_Avg': 'v.3-avg',
#                     'Uz_Avg': 'w.3-avg',
#                     'Ts_Avg': 't_sonic.3-avg',
#                     'CO2_Avg': 'co2.3-avg',
#                     'H2O_Avg': 'h2o.3-avg',
#                     'cell_press_Avg': 'p_irga.3-avg',
#                     'cell_tmpr_Avg': 't_irga.3-avg',
#                     'diag_sonic_Avg': 'diag_sonic.3-avg',
#                     'diag_irga_Avg': 'diag_irga.3-avg',
#                     'PTemp_C_Avg': 't_logger.0-avg',
#                     'BattV_Avg': 'v_logger.0-avg',
#                     'BattV_Min': 'v_logger.0-min',
#                     'Ux_Std': 'u.3-std',
#                     'Uy_Std': 'v.3-std',
#                     'Uz_Std': 'w.3-std',
#                     'Ts_Std': 't_sonic.3-std',
#                     'CO2_Std': 'co2.3-std',
#                     'H2O_Std': 'h2o.3-std',
#                     'cell_press_Std': 'p_irga.3-std',
#                     'cell_tmpr_Std': 't_irga.3-std',
#                     'diag_irga_Std': 'diag_sonic.3-std',
#                     'diag_sonic_Std': 'diag_irga.3-std',
#                     }
#                 df = df.drop(columns=cols_to_drop).rename(renaming_dict, axis='columns')
                
#                 # subset the data by instrument
#                 sonic_cols = ['timestamp', 'u.3-avg', 'v.3-avg', 'w.3-avg', 't_sonic.3-avg', 'diag_sonic.3-avg', 'u.3-std', 'v.3-std', 'w.3-std', 't_sonic.3-std', 'diag_sonic.3-std']
#                 irga_cols = ['timestamp', 'co2.3-avg', 'h2o.3-avg', 'p_irga.3-avg', 't_irga.3-avg', 'diag_irga.3-avg', 'co2.3-std', 'h2o.3-std', 'p_irga.3-std', 'diag_irga.3-std']
#                 logger_cols = ['timestamp', 't_logger.0-avg', 'v_logger.0-avg', 'v_logger.0-min']
                
#                 # pivot longer by expanding on height and stat suffixes
#                 ds = [df[sonic_cols], df[irga_cols], df[logger_cols]]

#                 for di, d in enumerate(ds):
#                     cols_to_stub = d.drop(columns='timestamp')
#                     i = ['timestamp', ['timestamp', 'stat']]
#                     j = ['stat', 'height']
#                     sep = ['-', '.']
#                     suffix = ['\w+', '\d+']
#                     stubnames = [[*set([colname.split(sep[0])[0] for colname in cols_to_stub])],
#                                  [*set([colname.split(sep[1])[0] for colname in cols_to_stub])]]
#                     for k in range(2):
#                         d = pd.wide_to_long(d, stubnames[k], i=i[k], j=j[k], sep=sep[k], suffix=suffix[k]).reset_index()
#                     d['site'] = 'UF3'
#                     d['replicate'] = 1

#                     ds[di] = d
                
#                 # upload (append) to database
#                 sonic = ds[0]
#                 irga = ds[1]
#                 logger = ds[2]

#                 sonic.to_sql('sonic_slow', con, if_exists='append', index=False)
#                 irga.to_sql('irga_slow', con, if_exists='append', index=False)
#                 logger.to_sql('logger_slow', con, if_exists='append', index=False)

In [373]:
upload_slowflux(db_fn)

Processing /Volumes/TempData/Bretfeld Mario/Chimney/Data/BB-UF/EC/3m/20200710/Converted/TOA5_2991.Flux30Min.dat: 100%|██████████| 294/294 [00:52<00:00,  5.61it/s]  


In [130]:


tables = dict(
    

    netrad_slow = dict(sw_in='REAL', sw_out='REAL', lw_in='REAL', lw_out='REAL', netrad='REAL', alb='REAL', tb='REAL'),

    ppfd_in_slow = dict(ppfd_in='REAL'),

    ppfd_out_slow = dict(ppfd_out='REAL'),

    sap_slow = dict(sapmv='REAL', stemtemp='REAL'),

    d_snow_slow = dict(dist_to='REAL', tcdist_to='REAL', d_snow='REAL', d_snow_taref='REAL'),

    tsn_slow = dict(tsn_tref='REAL', tsn='REAL'),

    burba_slow = dict(irgatemp='REAL', irgatempref='REAL'),

    pri_slow = dict(pri='REAL', down_532='REAL', down_570='REAL', up_532='REAL', up_570='REAL', ind_down='REAL', ind_up='REAL'),

    ta_slow = dict(ta='REAL'),

    rh_slow = dict(rh='REAL'),

    press_slow = dict(pa='REAL'),

    precip_slow = dict(p='REAL'),

    g_slow = dict(ts='REAL', g='REAL'),

    hydras_slow = dict(ts='REAL', swc='REAL'),

    wind_slow = dict(wd='REAL', ws='REAL'),
    
    logger_slow = dict(t_logger='REAL', v_logger='REAL'),

    # fast data
    ec_fast = dict(conv_fn='TEXT', raw_fn='TEXT'),

    # for qa/qc
    flags = dict(reviewed='INT', flagged='INT', comment='TEXT'),
    
    
)

for v in tables.values():
    v.update(key_cols)

con = sq.connect(db_fn)
for name, columns in tables.items():
    create_table(name, columns)